# Module 3: AI Agents with Semantic Kernel
## Building Intelligent AI Agents

### 1. Introduction to SK Agents
Agents in Semantic Kernel are AI-powered entities that can engage in conversations, make decisions, and execute tasks. They can work independently or collaborate in groups to achieve complex goals.


### 2. Creating Basic Agents


In [1]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory

# Create a kernel and add a chat service
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

# Create a simple agent with personality
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="Pirate",
    instructions="You are a friendly pirate who always speaks in pirate dialect and ends messages with a parrot sound."
)
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))


# Create chat history and helper function for interaction
chat = ChatHistory()

async def chat_with_agent(agent: ChatCompletionAgent, message: str):
    """Function to handle agent interaction"""
    chat.add_user_message(message)
    print(f"User: {message}")
    
    # Use streaming for responsive interaction
    chunks = []
    async for chunk in agent.invoke_stream(chat):
        chunks.append(chunk)
        print(chunk.content, end="", flush=True)  # Show response as it comes
    print("\n")  # New line after response
    
    # Add complete response to chat history
    complete_response = "".join([chunk.content for chunk in chunks])
    chat.add_assistant_message(complete_response)

print("Starting chat with Pirate Agent...")
    
    # Test different types of interactions
await chat_with_agent(agent, "Hello! Can you help me find treasure?")
await chat_with_agent(agent, "What's the best way to navigate at sea?")
await chat_with_agent(agent, "Tell me about your parrot!")

Starting chat with Pirate Agent...
User: Hello! Can you help me find treasure?
Ahoy there, matey! Ye be seekin' treasure, eh? Aye, I can help ye chart a course fer plunderin' riches! First, tell me what kind of treasure ye be after—gold doubloons, rare gems, or perhaps a hidden map? Let’s hoist the sails and set forth on this quest! Squawk! 🦜

User: What's the best way to navigate at sea?
Arrr, navigating the vast seas be a skill every pirate worth their salt must master! Here be a few tried-and-true ways to find yer way through the waves:

1. **Celestial Navigation**: Keep yer eyes on the stars, matey! By usin' the North Star (Polaris) and other celestial bodies, ye can chart yer course on clear nights.

2. **Compass**: A trusty compass be a pirate's best friend! It points toward the magnetic north, helpin' ye stay on course when the seas be rough and the skies be gray.

3. **Dead Reckoning**: This be the art of calculating yer position based on speed, time, and direction. Mark yer co

In [2]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

class WeatherPlugin:
    """Plugin for weather-related functions"""
    
    @kernel_function(description="Get the current weather for a location.")
    def get_weather(
        self,
        location: Annotated[str, "The city name"]
    ) -> str:
        # In real implementation, this would call a weather API
        return f"The weather in {location} is sunny and 22°C"

    @kernel_function(description="Get the weather forecast for next 3 days.")
    def get_forecast(
        self,
        location: Annotated[str, "The city name"]
    ) -> str:
        return f"3-day forecast for {location}: Sunny, Cloudy, Rain"

# Set up kernel with plugins
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

# Configure function auto-invocation
settings = kernel.get_prompt_execution_settings_from_service_id(service_id="agent")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Add plugin to kernel
kernel.add_plugin(WeatherPlugin(), plugin_name="weather")

# Create agent with access to plugins
agent = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="WeatherAssistant",
    instructions="""You help users with weather-related queries.
    Always aim to provide the most accurate and detailed information possible.
    When appropriate, combine current weather with forecast information.""",
    execution_settings=settings
)


chat = ChatHistory()

async def ask_weather(question: str):
    chat.add_user_message(question)
    print(f"User: {question}")

async for response in agent.invoke_stream(chat):
    print(response.content, end="", flush=True)
print("\n")

await ask_weather("What's the weather like in Seattle?")
await ask_weather("Should I pack an umbrella for my trip to London next week?")
await ask_weather("Compare the weather in New York and Tokyo.")

How can I assist you with weather-related information today?

User: What's the weather like in Seattle?
User: Should I pack an umbrella for my trip to London next week?
User: Compare the weather in New York and Tokyo.


In [3]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.open_ai import AzureAssistantAgent, OpenAIAssistantAgent

class ApprovalTerminationStrategy(TerminationStrategy):
    """
    Termination Strategy for Peer Review Discussion.

    This strategy evaluates the chat history to determine if the discussion
    should terminate based on achieving consensus or a clear approval.
    """

    async def should_agent_terminate(self, agent, history):
        """Evaluate termination condition."""
        return "approved" in history[-1].content.lower()


# Configuration for agents' personas and behavior
RESEARCHER_NAME = "AIResearcher"
RESEARCHER_INSTRUCTIONS = """
Role: AI Researcher
Objective: Refine and explain the technical details of a proposed model.
Actions:
- Answer questions concisely while focusing on scientific rigor.
- Propose refinements to improve the clarity and reproducibility of the paper.
- Avoid discussing unrelated topics; stay focused on the research at hand.
"""

REVIEWER_NAME = "PeerReviewer"
REVIEWER_INSTRUCTIONS = """
Role: Peer Reviewer
Objective: Evaluate the proposed research paper for clarity, significance, and rigor.
Actions:
- Identify ambiguities or potential improvements in methodology or claims.
- Approve the paper if it meets standards, using the word 'approved'.
- Suggest actionable refinements without rephrasing the entire paper.
"""


class ResearchToolsPlugin:
    """
    Plugin for Providing Research-Related Context and Tools.

    This plugin supports the agents by offering functionality for citing references,
    summarizing research papers, and validating datasets.
    """

    @kernel_function(description="Provides a formatted citation for a given paper.")
    def cite_paper(
        self, title: Annotated[str, "The title of the paper."],
        author: Annotated[str, "The author of the paper."],
        year: Annotated[int, "The year of publication."]
    ) -> Annotated[str, "Returns the citation in APA format."]:
        """Generate a citation."""
        return f"{author} ({year}). {title}. Journal of AI Research."

    @kernel_function(description="Summarizes the key contributions of a paper.")
    def summarize_paper(
        self, abstract: Annotated[str, "The abstract of the paper."]
    ) -> Annotated[str, "Returns a concise summary of the paper's contributions."]:
        """Summarize a research paper's contributions."""
        return f"Key Contributions: {abstract[:200]}..."  # Truncated for brevity.


def initialize_kernel_with_research_tools(service_id: str) -> Kernel:
    """
    Initialize a Semantic Kernel instance with Azure Chat Completion and research tools.

    Args:
        service_id (str): Identifier for the chat service.

    Returns:
        Kernel: Configured Semantic Kernel instance.
    """
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    kernel.add_plugin(plugin=ResearchToolsPlugin(), plugin_name="research_tools")
    return kernel


# Initialize the Kernel and agents
kernel = initialize_kernel_with_research_tools("peerreviewer")
settings = kernel.get_prompt_execution_settings_from_service_id(service_id="peerreviewer")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

agent_researcher = ChatCompletionAgent(
    service_id="peerreviewer",
    kernel=kernel,
    name=RESEARCHER_NAME,
    instructions=RESEARCHER_INSTRUCTIONS,
    execution_settings=settings,
)

agent_reviewer = await AzureAssistantAgent.create(
    service_id="researcher",
    kernel=Kernel(),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)

# Define group chat with termination strategy
chat = AgentGroupChat(
    agents=[agent_researcher, agent_reviewer],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[agent_reviewer], maximum_iterations=10
    ),
)

# User initiates the conversation
user_input = "Refine the methodology section of the proposed AI model paper."
await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
print(f"# {AuthorRole.USER}: '{user_input}'")

# Process responses from the agents
async for response in chat.invoke():
    print(f"# {response.role} - {response.name or '*'}: '{response.content}'")

print(f"# Chat Complete: {chat.is_complete}")


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [4]:
"""Fake user generator.

This module generates fake users and stores them in a easydb database.
These user are generated using the faker library with a little bit of LLM magic.

Goal is having a user base for a virtual forum.

fields:

real_name:
user_name:
gender:
age:
alignment:
bio:
email:
interests (real life):
interests (virtual, online):
political_views:
social_views:

"""

import random
from datetime import date

from faker import Faker
from pydantic import BaseModel, Field
from rich.console import Console
from schwarm.core.schwarm import Schwarm
from schwarm.models.types import Agent, ContextVariables, Result
from schwarm.utils.settings import APP_SETTINGS
from tinydb import TinyDB

fake = Faker()
console = Console()
console.clear()
APP_SETTINGS.DATA_FOLDER = ""

db = TinyDB(f"{APP_SETTINGS.DATA_FOLDER}/db.json")


class User(BaseModel):
    """User model."""

    real_name: str = Field(..., title="Real Name")
    user_name: str = Field(..., title="User Name")
    gender: str = Field(..., title="gender")
    birthday: str = Field(..., title="Birthday")
    age: int = Field(..., title="Age")
    alignment: str = Field(..., title="Alignment")
    bio: str = Field(..., title="Bio")
    job: str = Field(..., title="Job")
    email: str = Field(..., title="Email")
    interests_real: str = Field(..., title="Interests (real life)")
    interests_virtual: str = Field(..., title="Interests (virtual, online)")
    political_views: str = Field(..., title="Political Views")
    social_views: str = Field(..., title="Social Views")


# Agents

user_generator = Agent(name="User Generator")
bio_generator = Agent(name="Biography Generator")

# Instructions


def instruction_user_generator(context_variables: ContextVariables) -> str:
    instruction = """
    You are a helpful agent specialized in choosing the right tool for the task at hand.
    Task: Generating fake users for a virtual forum.
    """
    return instruction


def instruction_bio_generator(context_variables: ContextVariables) -> str:
    instruction = """
    You are a helpful agent specialized in choosing the right tool for the task at hand.
    Task: Generate a bio for a fake user.
    """
    return instruction


user_generator.instructions = instruction_user_generator
bio_generator.instructions = instruction_bio_generator


# Functions


def get_random_alignment() -> str:
    """Get a random alignment."""
    algnmt = random.randint(1, 9)
    alignment = "True Neutral"
    if algnmt == 1:
        alignment = "Lawful Good"
    elif algnmt == 2:
        alignment = "Neutral Good"
    elif algnmt == 3:
        alignment = "Chaotic Good"
    elif algnmt == 4:
        alignment = "Lawful Neutral"
    elif algnmt == 5:
        alignment = "True Neutral"
    elif algnmt == 6:
        alignment = "Chaotic Neutral"
    elif algnmt == 7:
        alignment = "Lawful Evil"
    elif algnmt == 8:
        alignment = "Neutral Evil"
    elif algnmt == 9:
        alignment = "Chaotic Evil"
    return alignment


def get_gender() -> str:
    r = random.randint(0, 100)
    gender = "M"
    if r > 80:
        gender = "F"
    return gender


def get_birthdate() -> date:
    """Get a random birthdate."""
    return fake.date_of_birth(minimum_age=18, maximum_age=80)


def transfer_user_data_to_biography_generator(context_variables: ContextVariables) -> Result:
    """Generate a bio for a user."""
    # Generate some fields with faker

    # Gender with a 80% chance of being male and 20% of being female

    profile = fake.profile(sex=get_gender())  # type: ignore
    birthdate = get_birthdate()

    user = User(
        real_name=str(profile["name"]),
        user_name="",
        gender=str(profile["sex"]),
        birthday=str(birthdate),
        age=2024 - birthdate.year,  # type: ignore
        alignment=get_random_alignment(),  # type: ignore
        bio="",
        job=str(profile["job"]),
        email=str(profile["mail"]),
        interests_real="",
        interests_virtual="",
        political_views="",
        social_views="",
    )
    context_variables["user"] = user
    return Result(value=f"{user}", context_variables=context_variables, agent=bio_generator)


def transer_user_bio_to_user_generator(
    context_variables: ContextVariables,
    user_name: str,
    biography: str,
    interests_real: str,
    interests_virtual: str,
    political_views: str,
    social_views: str,
) -> Result:
    """Save user bio to database.

    The bio should be a short, but informative text that gives a clear picture of the user.
    It should fit the alignment of the user, and the rest of the profile, like age and job

    Arguments:
        user_name: The user name of the user. The more creative the better.
        biography: The biography of the user. Short, but informative. Should give a clear picture of the user.
        interests_real: The real life interests of the user. comma seperated tags.
        interests_virtual: The virtual interests of the user. comma seperated tags.
        political_views: The political views of the user. comma seperated tags.
        social_views: The social views of the user. comma seperated tags.
    """
    user = context_variables["user"]
    if "count" in context_variables:
        context_variables["count"] += 1
    else:
        context_variables["count"] = 1
    count = context_variables["count"]
    if isinstance(user, User):
        user.bio = biography
        user.user_name = user_name
        user.interests_real = interests_real
        user.interests_virtual = interests_virtual
        user.political_views = political_views
        user.social_views = social_views
    context_variables["user"] = user
    db.insert(user.dict())  # type: ignore
    console.print(f"User {context_variables['count']}:\n{biography}")
    return Result(value=f"User #{count}: {user}", context_variables=context_variables, agent=user_generator)


user_generator.functions = [transfer_user_data_to_biography_generator]
bio_generator.functions = [transer_user_bio_to_user_generator]

response = Schwarm().quickstart(user_generator, "Start generating!", mode="auto")


/workspaces/wd-ai-hackathon/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'error_msg_templates' has been removed
  warnings.warn(message, UserWarning)


2024-11-23 14:48:36.671 | INFO     | schwarm.core.schwarm:__init__:38 - Schwarm instance initialized
2024-11-23 14:48:36.672 | DEBUG    | schwarm.core.logging:wrapper:141 - START schwarm.core.schwarm.Schwarm.quickstart:
  args: <schwarm.core.schwarm.Schwarm object at 0x7aba917baf30>, Agent(name='User Generator', model='gpt-4o', description='', instructions=<function instruction_user_generator at 0x7aba914be5c0>, functions=[<...(+470 chars)
  kwargs: 
2024-11-23 14:48:36.674 | DEBUG    | schwarm.core.logging:wrapper:141 - START schwarm.core.schwarm.Schwarm.run:
  args: <schwarm.core.schwarm.Schwarm object at 0x7aba917baf30>, Agent(name='User Generator', model='gpt-4o', description='', instructions=<function instruction_user_generator at 0x7aba914be5c0>, functions=[<...(+851 chars)
  kwargs: 
2024-11-23 14:48:36.675 | INFO     | schwarm.core.schwarm:run:142 - Disabling console logging


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:202                                                                                  │
│                                                                                                  │
│   199 user_generator.functions = [transfer_user_data_to_biography_generator]                     │
│   200 bio_generator.functions = [transer_user_bio_to_user_generator]                             │
│   201                                                                                            │
│ ❱ 202 response = Schwarm().quickstart(user_generator, "Start generating!", mode="auto")          │
│   203                                                                                            │
│                                                                                                  │
│ /workspaces/wd-ai-hackathon/.venv/lib/python3.12/site-packages/schwarm/core/logging.py:147 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   144 │   │   │                                                                                  │
│   145 │   │   │   try:                                                                           │
│   146 │   │   │   │   # Execute the function                                                     │
│ ❱ 147 │   │   │   │   result = func(*args, **kwargs)                                             │
│   148 │   │   │   │                                                                              │
│   149 │   │   │   │   # Calculate execution time                                                 │
│   150 │   │   │   │   execution_time = time.time() - start_time                                  │
│                                                                                                  │
│ /workspaces/wd-ai-hackathon/.venv/lib/python3.12/site-packages/schwarm/core/schwarm.py:79 in     │
│ quickstart                                                                                       │
│                                                                                                  │
│    76 │   │   if context_variables is None:                                                      │
│    77 │   │   │   context_variables = {}                                                         │
│    78 │   │   if mode == "auto":                                                                 │
│ ❱  79 │   │   │   return self.run(                                                               │
│    80 │   │   │   │   agent,                                                                     │
│    81 │   │   │   │   messages=[Message(role="user", content=input_text)],                       │
│    82 │   │   │   │   context_variables=context_variables,                                       │
│                                                                                                  │
│ /workspaces/wd-ai-hackathon/.venv/lib/python3.12/site-packages/schwarm/core/logging.py:147 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   144 │   │   │                                                                                  │
│   145 │   │   │   try:                                                                           │
│   146 │   │   │   │   # Execute the function                                                     │
│ ❱ 147 │   │   │   │   result = func(*args, **kwargs)                                             │
│   148 │   │   │   │                                                                              │
│   149 │   │   │   │   # Calculate execution time           